# 공통

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

### 2018-08-13
- Baseline 구축
- X : MACD, MACD>0 & MACDDiff>0 (Long&Short), returns, ewm
  - For loop돌 때 ewm 반복된 할당
- Y : Class of returns
  - shift 오류
- Train, Test Split
- Standard Scaler로 Scale 조정
- GNB, KNN, RF
  - Predict 평균 50
- Grid Search
  - best estimator로 재예측

### 2018-08-16
- X : 5~30 Moving Average, minmaxVolume, minmaxClose
- Y : 수익률 - 수익률.rolling(5).mean()
  - shift 오류
- GNB, KNN, RF
  - Predict 평균 50~60
- Grid Search
  - 시간 상 KeyboardInterrupt

### 2018-08-23
- fix_yahoo_finance에서 주식 데이터 read
- X : Close Moving Average, Volume Moving Average, Historical Volatility
- Y : Class of
 - Y1 = Close - Close_MA_7
 - Y2 = Returns - Returns_MA_7
 - Y3 = Close.rolling(7).sum().pct_change()
   - shift 오류
- 2012-01-01 : ~
- XGBoost 첫 사용
- modelfit 함수, useTrainCV, dMatrix 사용
- Feature Importance 계산
- Train에서 높은 Fitting을 보여주지만 Test에서 별다른 성과를 얻지 못함

In [ ]:
def modelfit(alg, dtrain, predictors, target, useTrainCV = True, cv_folds = 5, early_stopping_rounds = 50):
    
    start = time.time()
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_stdv=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("Model Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
                    
    imp_fear_name = pd.Series(predictors)[alg.feature_importances_ > 0.01]
    imp_fear_name.index = range(len(imp_fear_name))
    imp_fear = pd.concat((imp_fear_name, 
               pd.Series(alg.feature_importances_[alg.feature_importances_ > 0.01])),axis=1).sort_values(by=1,ascending=True)
    imp_fear.index = range(len(imp_fear))
    imp_fear.columns = ["feature_name", "feature_importance"]
#     {i:j for i,j in zip(imp_fear.index, imp_fear["feature_name"])}
    imp_fear.plot(y='feature_importance', x='feature_name', kind='barh', legend=False, figsize=(8,5))
    
#     feat_imp = pd.Series(alg.feature_importances_).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances', figsize=(200,5))
#     plt.ylabel('Feature Importance Score')
    
    elapse = time.time() - start
    print("elapse time : {}".format(round(elapse,2)))
    
    return alg, imp_fear

### 2018-08-26
- X : Close Moving Average, Volume Moving Average, Historical Volatility
- Y : Class of
 - Y1 = Close - Close_MA_7
 - Y2 = Returns - Returns_MA_7
 - Y3 = Close.rolling(7).sum().pct_change()
   - shift 오류
- 예측 결과로 Simulation실시
- Y2와 XGBClassifier로 예측 실시
  - Predict 80% up
  - 허나 이는 잘못된 결과임을 곧 알게된다..

### 2018-09-10
- 고대교수님 세미나 준비
- X, Y 변화 없음
- GNB, KNN, RF, DT, LOG, SVM, XGB 사용
- time 계산 후 Feature Selection으로 시간이 얼마나 단축되는지 계산

### 2018-09-13
- modelfit함수 수정
- 기타 잘못된 방법으로 모델 학습 진행

In [ ]:
def modelfit2(alg, dtrain, test, predictors, target, useTrainCV = True, 
             cv_folds = 5, early_stopping_rounds = 50, top = 10, 
             figure = True, model_report = True):
    
    start = time.time()
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_stdv=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    test_predictions = alg.predict(test[predictors])
    test_predprob = alg.predict_proba(test[predictors])[:,1]
        
    if model_report:
        #Print model report:
        print("Model Report")
        print("Accuracy (Train): %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
        print("Accuracy (Test): %.4g" % metrics.accuracy_score(test[target].values, test_predictions))
        print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
        print("AUC Score (Test): %f" % metrics.roc_auc_score(test[target], test_predprob))
                    
    imp_fear_name = pd.Series(predictors)[alg.feature_importances_ > 0.01]
    imp_fear_name.index = range(len(imp_fear_name))
    imp_fear = pd.concat((imp_fear_name, 
               pd.Series(alg.feature_importances_[alg.feature_importances_ > 0.01])),axis=1).sort_values(by=1,ascending=True)
    imp_fear.index = range(len(imp_fear))
    imp_fear.columns = ["feature_name", "feature_importance"]
#     {i:j for i,j in zip(imp_fear.index, imp_fear["feature_name"])}
    if figure:
        if top:
            imp_fear.iloc[-top:].plot(y='feature_importance', x='feature_name', kind='barh', legend=False, figsize=(8,5))
        else:
            imp_fear.plot(y='feature_importance', x='feature_name', kind='barh', legend=False, figsize=(8,5))
    
#     feat_imp = pd.Series(alg.feature_importances_).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances', figsize=(200,5))
#     plt.ylabel('Feature Importance Score')
    
    elapse = time.time() - start
    print("elapse time : {} sec".format(round(elapse,2)))
    
    return alg, imp_fear, elapse

### 2018-09-17
- 잘못된 것을 깨닫고 학습 진행
- Cross Val Predict 실습 진행
- Y shift 수정
- MACD, Stochastic, RSI로 위기를 모면하고자 함수화 진행
- 예측률 떡락 (66%)

In [ ]:
def modelfit3(alg, dtrain, test, predictors, target, useTrainCV = True, 
             cv_folds = 5, early_stopping_rounds = 50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_stdv=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target], eval_metric='auc')
        
    #Predict training set:
    test_predictions = alg.predict(test[predictors])
    test_predprob = alg.predict_proba(test[predictors])[:,1]
        
    fi_dict = {feature : importance for feature, importance in zip(predictors, alg.feature_importances_)}

    return alg, fi_dict, test_predictions, test_predprob

### 2018-09-18
- TestAccuracy 함수 제작
- 예측률 77%
- 주식 소거 기준 제공
  - del_li = [key for key, df in raw_data.items() if len(df) < 252*2]
  - del_li2 = ['BGF', 'HDC', 'SK디스커버리', '대우조선해양', '대한항공', '동아쏘시오홀딩스', '동양', '롯데지주', '오리온홀딩스', '우리은행', '쿠쿠홀딩스', '팬오션', '한국타이어월드와이드', '한라홀딩스', '현대중공업', '효성'] 

In [ ]:
def TestAccuracy(train, test, predictors, target, alg):
    score_set = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'auc', 'f1'])
    for i in raw_data.keys():
        X_train = train[i][predictors].values
        y_train = train[i][target].values
        X_test = test[i][predictors].values
        y_test = test[i][target].values
        a=[]
        alg.fit(X_train, y_train)
        y_pred = alg.predict(X_test)
        a.append(metrics.accuracy_score(y_test, y_pred))
        value_1=cross_val_predict(alg, X_train, y_train, cv=5,method="predict_proba")
        a.append(metrics.precision_score(y_test, y_pred))
        a.append(metrics.recall_score(y_test, y_pred))
        fpr, tpr, thresholds = metrics.roc_curve(y_train, value_1[:,1])
        a.append(metrics.auc(fpr, tpr))
        a.append(metrics.f1_score(y_test,y_pred).mean())
        score_set.loc[i] = a
    return score_set

In [ ]:
def temp_calc_feature_importance(alg, train, test, predictors, target):
    test_result = {}
    fi_dict = {}
    for i in train.keys():
        model, fear_importance = modelfit(alg, train[i], predictors, target, 
                                                  cv_folds=5, early_stopping_rounds=20,
                                                  figure=False, model_report=False)
        pred = model.predict(test[i][predictors])
        # Calculating Score
        test_result[i] = np.array([
            metrics.accuracy_score(pred, test[i][target]),
            metrics.f1_score(pred, test[i][target]),
            metrics.precision_score(pred, test[i][target]),
            metrics.recall_score(pred, test[i][target]),
            metrics.roc_auc_score(pred, test[i][target])
        ])
        fi_dict[i] = {feature:value for feature,value in fear_importance.iloc[-10:].values}
    return test_result, fi_dict

### 2018-10-08
- Kospi의 수익률을 Feature로 추가
- Kaggle의 Sigma competition을 참고, 각종 수익률에 대한 feature를 고려
- Y의 Predict 기간을 조정
- 학습 준비 파일을 제공 (20181013version)
- 2010부터의 데이터를 제공 (20181008version)

### 2018-10-13
- moon_code로 실험

### 2018-10-22
- 뜬금없이 Panel로 노선 변경

In [2]:
def get_multi_index(col_name):
    return pd.MultiIndex.from_arrays([[col_name]*len(panel.loc['Close']),
                                      panel.index.get_level_values(1).unique().tolist()])

def get_multi_index(col_name):
    return pd.MultiIndex.from_product([[col_name],list(raw_data.keys())])

def add_feature(q, col_name, expr):
    ind = get_multi_index(col_name)
    try:
        df = pd.DataFrame(expr.values, columns=ind, index=q.index)
    except AttributeError:
        df = pd.DataFrame(expr, columns=ind, index=q.index)
    q = pd.concat((q, df), axis=1)
    return q

### 2018-10-24
- xgb_modelfit 사용
- portfolio 제작

In [ ]:
def get_col_df(df_name, fillna=None):
    df = pd.DataFrame()
    for stock in model_set.keys():
        df = pd.concat((df,validation_[stock][df_name]), axis=1)
    df.columns = list(model_set.keys())
    if fillna == 'mean':
        df = df.fillna(df.mean())
    else:
        df = df.fillna(0)
    col = pd.MultiIndex.from_product([['{}'.format(df_name)],df.columns])
    df.columns = col
    return df

In [ ]:
import copy

def modelfit(alg, train, test, X, y):
    alg = copy.deepcopy(alg)
    xgb_params = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(train[X].values, label=train[y].values)
    cvresult=xgb.cv(xgb_params, xgtrain,
                    num_boost_round=alg.get_params()['n_estimators'],
                    nfold=5, early_stopping_rounds=50,
                    show_stdv=False)
    alg.set_params(n_estimators=cvresult.shape[0])
    alg.fit(train[X], train[y], eval_metric='auc')
    dtrain_predictions = alg.predict(train[X])
    dtrain_predprob = alg.predict_proba(train[X])[:,1]
    dtest_predictions = alg.predict(test[X])
    dtest_predprob = alg.predict_proba(test[X])[:,1]
    print("Train Accuracy : {:.4f}".format(metrics.accuracy_score(
                            train[y].values, dtrain_predictions)))
    print("Train AUC Score : {:.4f}".format(metrics.roc_auc_score(
                            train[y].values, dtrain_predprob)))
    print("Test  Accuracy : {:.4f}".format(metrics.accuracy_score(
                            test[y].values, dtest_predictions)))
    print("Test  AUC Score : {:.4f}".format(metrics.roc_auc_score(
                            test[y].values, dtest_predprob)))
    return alg

### 2018-11-07
- 작업 준비
- Y3한정 90% 돌파

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pickle
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)
ks11 = pd.read_csv('ks11.csv')
ks11.index = ks11.Date
del ks11['Date']

copy_keys = list(raw_data.keys())
for stock in copy_keys:
    if len(raw_data[stock].loc[:'2013-01-01']) < 1:
        del raw_data[stock]
        
ks11 = ks11.drop(ks11.index[np.where(ks11.Volume==0)[0]])
        
for key, df in raw_data.items():
    raw_data[key] = df.drop(df.index[np.where(df.Volume==0)[0]], axis=0)
    
# data = ks11.copy()
# data = raw_data['삼성전자'].copy()
# data = raw_data['NAVER'].copy()
# data = raw_data['LG화학'].copy()
data = raw_data['미래에셋대우'].copy()

data["pct_change"] = data.Close.pct_change()
for i in range(3, 31, 2):
    data["close_ma_{}".format(i)] = data.Close.rolling(window = i).mean()
for i in [7, 21, 30, 60, 90, 91, 100, 120]:
    data["his_vol_{}".format(i)] = data["pct_change"].rolling(window = i).std()*(252**0.5)
for i in [1, 3, 5, 7, 10, 12, 14]:
    data['returnsCC{}'.format(i) ]= np.log(data.Open / data.Close.shift(i))
    data['returnsOO{}'.format(i) ] = np.log(data.Open / data.Open.shift(i))
    data['returnsCC{}'.format(i) ]= np.log(data.Close / data.Close.shift(i))
    data['returnsOC{}'.format(i) ] = np.log(data.Close / data.Open.shift(i))
for i in [1, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50,
          55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]:
    for shift in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        data['Y1_{}_s{}'.format(i, shift)] = (data.Close - data.Close.shift(shift).rolling(i).mean())
        data['Y3_{}_s{}'.format(i, shift)] = (data.Close.rolling(i).mean() - data.Close.shift(shift).rolling(i).mean())
        data['sh{}_Y1_{}'.format(shift, i)] = np.where((data['Y1_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        data['sh{}_Y3_{}'.format(shift, i)] = np.where((data['Y3_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        
data_ = data.loc['2018-07-01':'2018-10-01']
data = data.loc['2013-07-01':'2018-07-01']

X_li = [i for i in data.columns if (i.find('sh') == -1) if (i.find('Volume') == -1)]

X_li2 = [i for i in data.columns 
         if (i.find('sh') == -1) 
         if (i.find('Volume') == -1)
         if (i.find('Y') == -1)]

%run moon_code
%run xgb_modelfit

import copy

train, test = train_test_split(data, 0.8, dtrain=True)

y_li = [i for i in data.columns if i.find('sh') != -1]

def modelfit(alg, train, test, X, y):
    alg = copy.deepcopy(alg)
    xgb_params = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(train[X].values, label=train[y].values)
    cvresult=xgb.cv(xgb_params, xgtrain,
                    num_boost_round=alg.get_params()['n_estimators'],
                    nfold=5, early_stopping_rounds=50,
                    show_stdv=False)
    alg.set_params(n_estimators=cvresult.shape[0])
    alg.fit(train[X], train[y], eval_metric='auc')
    dtrain_predictions = alg.predict(train[X])
    dtrain_predprob = alg.predict_proba(train[X])[:,1]
    dtest_predictions = alg.predict(test[X])
    dtest_predprob = alg.predict_proba(test[X])[:,1]
#     print("Train Accuracy : {:.4f}".format(metrics.accuracy_score(
#                             train[y].values, dtrain_predictions)))
#     print("Train AUC Score : {:.4f}".format(metrics.roc_auc_score(
#                             train[y].values, dtrain_predprob)))
#     print("Test  Accuracy : {:.4f}".format(metrics.accuracy_score(
#                             test[y].values, dtest_predictions)))
#     print("Test  AUC Score : {:.4f}".format(metrics.roc_auc_score(
#                             test[y].values, dtest_predprob)))
    return alg

for idx, stock in enumerate(raw_data.keys()):
        
    if idx < a : 
        continue
    if idx > b :
        break
        
    print(stock)
    data = raw_data[stock].copy()

    data["pct_change"] = data.Close.pct_change()
    for i in range(3, 31, 2):
        data["close_ma_{}".format(i)] = data.Close.rolling(window = i).mean()
    for i in [7, 21, 30, 60, 90, 91, 100, 120]:
        data["his_vol_{}".format(i)] = data["pct_change"].rolling(window = i).std()*(252**0.5)
    for i in [1, 3, 5, 7, 10, 12, 14]:
        data['returnsCC{}'.format(i) ]= np.log(data.Open / data.Close.shift(i))
        data['returnsOO{}'.format(i) ] = np.log(data.Open / data.Open.shift(i))
        data['returnsCC{}'.format(i) ]= np.log(data.Close / data.Close.shift(i))
        data['returnsOC{}'.format(i) ] = np.log(data.Close / data.Open.shift(i))
    for i in [1, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50,
              55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]:
        for shift in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
            data['Y1_{}_s{}'.format(i, shift)] = (data.Close - data.Close.shift(shift).rolling(i).mean())
            data['Y3_{}_s{}'.format(i, shift)] = (data.Close.rolling(i).mean() - data.Close.shift(shift).rolling(i).mean())
            data['sh{}_Y1_{}'.format(shift, i)] = np.where((data['Y1_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
            data['sh{}_Y3_{}'.format(shift, i)] = np.where((data['Y3_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)

    data_ = data.loc['2018-07-01':'2018-10-01']
    data = data.loc['2013-07-01':'2018-07-01']

    X_li = [i for i in data.columns if (i.find('sh') == -1) if (i.find('Volume') == -1)]

    train, test = train_test_split(data, 0.8, dtrain=True)

    y_li = [i for i in data.columns if i.find('sh') != -1]

    model_set = {}
    for y in y_li:
        model = modelfit(alg, train, test, X_li, y)
        model_set[y] = model
        
    try:    
        y1 = pd.DataFrame()
        y3 = pd.DataFrame()
        for idx, y in enumerate(y_li):
            acc = metrics.accuracy_score(test[y], model_set[y].predict(test[X_li]))
            auc = metrics.roc_auc_score(test[y], model_set[y].predict(test[X_li]))
            res = pd.DataFrame([acc,auc], index=['accuracy', 'AUC'], columns=[y])
            if idx%2==0:
                y1 = pd.concat((y1, res), axis=1)
            else:
                y3 = pd.concat((y3, res), axis=1)

        Y1 = np.array([])
        for i in range(int(270/10)):
            i *= 10
            if i == 0:
                Y1 = y1.values[:,i:i+10]
            else:
                Y1 = np.vstack((Y1, y1.values[:,i:i+10]))
    except ValueError:
        continue

    Y3 = np.array([])
    for i in range(int(270/10)):
        i *= 10
        if i == 0:
            Y3 = y3.values[:,i:i+10]
        else:
            Y3 = np.vstack((Y3, y3.values[:,i:i+10]))

    Y = np.hstack((Y3, Y1))

    y1.to_excel('./20181105_result/y1_{}.xlsx'.format(stock))
    y3.to_excel('./20181105_result/y3_{}.xlsx'.format(stock))
    pd.DataFrame(Y).to_excel('./20181105_result/Y_{}.xlsx'.format(stock))

    with open('./20181105_result/model_set_{}_20181031.pickle'.format(stock), 'wb') as handle:
        pickle.dump(model_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 2018-11-21
- Regression >> Classification으로 넘어가기 위한 준비 단계